In [1]:
import findspark
findspark.init('/home/brunosouza/Documents/spark')

In [2]:
import sys
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [3]:
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [4]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

In [5]:
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)


In [6]:
wordCounts = words.groupBy("word").count()

In [7]:
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

In [8]:
# ./bin/spark-submit Net-cat-streaming-Demo.py localhost 9999